In [67]:
import pandas as pd
import numpy as np

data = pd.read_csv("data.csv")

In [99]:
pd.set_option('display.max_colwidth', None)

In [69]:
def limpiar_puntos(texto):
    if pd.isna(texto): return texto
    partes = str(texto).split('.')
    if len(partes) > 2:
        # Une la primera parte con la segunda mediante un punto, ignora el resto
        return f"{partes[0]}.{partes[1]}"
    return texto

data['start'] = data['start'].apply(limpiar_puntos).astype(float)
data['end'] = data['end'].apply(limpiar_puntos).astype(float)

In [70]:
# limpieza de textos vacíos
data['text'] = data['text'].str.strip()
data = data.replace('', np.nan).dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7191 entries, 0 to 7527
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  7191 non-null   int64  
 1   id_video    7191 non-null   object 
 2   start       7191 non-null   float64
 3   end         7191 non-null   float64
 4   text        7191 non-null   object 
dtypes: float64(2), int64(1), object(2)
memory usage: 337.1+ KB


In [71]:
data = data[data['id_video'] != 'zC_kiEcthOMpIcQFZzC3Kc' ]


In [114]:

# Limpiamos el formato extraño de puntos antes de empezar
data['start'] = pd.to_numeric(data['start'].astype(str).str.replace(r'\.(?=\d{2}\.)', '', regex=True), errors='coerce')
data['end']   = pd.to_numeric(data['end'].astype(str).str.replace(r'\.(?=\d{2}\.)', '', regex=True), errors='coerce')


In [115]:
# intervalo de merge de 2 segundos: start < 2s

df = data.copy()

def segmentacion_estricta(df):
    bloques = []
    texto_acumulado = []
    tiempo_inicio = None
    
    signos_cierre = ('.', '!', '?', '…')

    for i, row in df.iterrows():
        # Guardamos el inicio del primer segmento del bloque
        if tiempo_inicio is None:
            tiempo_inicio = row['start']
        
        texto_actual = str(row['text']).strip()
        texto_acumulado.append(texto_actual)
        
        # SI el texto termina en puntuación, cerramos el bloque
        if texto_actual.endswith(signos_cierre):
            bloques.append({
                'id_video': row['id_video'],
                'start': tiempo_inicio,
                'end': row['end'],
                'text': " ".join(texto_acumulado)
            })
            # Reiniciamos para el siguiente bloque
            texto_acumulado = []
            tiempo_inicio = None
            
    # Manejo de texto huérfano (si el video termina sin punto final)
    if texto_acumulado:
        bloques.append({
            'id_video': df.iloc[-1]['id_video'],
            'start': tiempo_inicio,
            'end': df.iloc[-1]['end'],
            'text': " ".join(texto_acumulado)
        })
        
    return pd.DataFrame(bloques)



In [116]:
# Uso
df_final = segmentacion_estricta(df)
def generar_url(row):
    # Convertimos a entero porque YouTube no procesa milisegundos en la URL
    segundos = int(row['start'])
    return f"https://www.youtube.com/watch?v={row['id_video']}&t={segundos}s"

# Creamos la nueva columna
df_final['link'] = df_final.apply(generar_url, axis=1)

In [119]:
tokensearch = r'(?=.*japonesa)(?=.*leyenda)'
res = df_final[df_final['text'].str.contains(tokensearch, regex=True, case=False, na=False)]
res
#print(res.iloc[0]['text'])

,id_video,start,end,text,link
1571,WQj3DXvVP10,819.82,826.56,"Hay una vieja leyenda nipona, una vieja leyenda japonesa que habla del honor que conversamos hace un rato.",https://www.youtube.com/watch?v=WQj3DXvVP10&t=819s
1598,WQj3DXvVP10,904.34,906.64,Vieja leyenda japonesa que habla de la paz.,https://www.youtube.com/watch?v=WQj3DXvVP10&t=904s
